In [1]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
parentdir = os.path.dirname(parentdir)
sys.path.insert(0,parentdir)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import sys
sys.path.append('/content/drive/My Drive/D2FM')
module_path = '/content/drive/My Drive/D2FM'
if module_path not in sys.path:
    sys.path.append(module_path)

In [4]:
!pip install pykalman

In [5]:
import models.ddfm

In [6]:
import numpy as np
import pandas as pd
import tensorflow as tf
from typing import Tuple
from tensorflow import keras
from tensorflow.keras import layers
from statsmodels.tsa.statespace.dynamic_factor_mq import DynamicFactorMQ as DFM
from sklearn.model_selection import ParameterGrid

In [7]:
#from google.colab import files
#uploaded = files.upload()

In [8]:
from models.ddfm import DDFM
from synthetic_dgp.simulate import SIMULATE
import warnings
warnings.filterwarnings("ignore")
import random

In [9]:
df_train = pd.read_csv('/content/drive/MyDrive/D2FM/sfr_train.csv')
df_test=pd.read_csv('/content/drive/MyDrive/D2FM/sfr_train.csv')
df_train_idx=df_train.set_index("index")
df_test_idx=df_train.set_index("index")

## Se corre el algoritmo 1 para distintas grillas de estructura de encoders, entradas de lags, considerando que la verdadera cantidad de factores es 7

In [10]:
r_true=7
structure_encoders = [
    (r_true * 4, r_true * 2, r_true),
    (r_true * 6, r_true * 4, r_true),
    (r_true * 12, r_true * 6, r_true)
]

# Define the lags to be used as input.
lags_inputs = list(range(3))

# Placeholder for storing models if needed.
model_info = []
# Iterate over each structure_encoder configuration.
for structure_encoder in structure_encoders:
    # Iterate over each lag value.
    for lags_input in lags_inputs:
        # Initialize the DDFM with the current configuration.
        ddfm = DDFM(df_train_idx, lags_input=lags_input, structure_encoder=structure_encoder, optimizer='Adam',
                    factor_oder=1, use_bias=False, link='tanh', epochs=100, max_iter=1000)

        # Fit the model.
        ddfm.fit()
        print(model_info.append((ddfm, structure_encoder, lags_input)))
        #predicted=ddfm.predict()

for model, encoder_config, lag in model_info:
    print(f"Model with encoder {encoder_config} , lag {lag}")


@Info - Note: Sorting data.
19/19 [==============================] - 0s 5ms/step
@Info: Convergence achieved in 9 iterations - new loss: 0.5688758629767554 - delta: 0.00040501166274600634 < 0.0005
None
@Info - Note: Sorting data.
18/18 [==============================] - 0s 2ms/step
None
@Info - Note: Sorting data.
18/18 [==============================] - 0s 4ms/step
@Info: Convergence achieved in 9 iterations - new loss: 0.561019267302351 - delta: 0.0002805175683906425 < 0.0005
None
@Info - Note: Sorting data.
19/19 [==============================] - 0s 3ms/step
@Info: Convergence achieved in 8 iterations - new loss: 0.5671230690723299 - delta: 0.00031357047570418367 < 0.0005
None
@Info - Note: Sorting data.
18/18 [==============================] - 0s 2ms/step
@Info: Convergence achieved in 7 iterations - new loss: 0.5612291745400497 - delta: 0.0004773610331235264 < 0.0005
None
@Info - Note: Sorting data.
18/18 [==============================] - 0s 2ms/step
@Info: Convergence achieved 

In [27]:
deep_dyn_fact_mdl = DDFM(df_train_idx, structure_encoder=(84, 42, 7), lags_input=2, factor_oder=1,
                             use_bias=False, link='relu', max_iter=100)
deep_dyn_fact_mdl.fit()


@Info - Note: Sorting data.
18/18 [==============================] - 0s 3ms/step
@Info: Convergence achieved in 7 iterations - new loss: 0.5775821049330419 - delta: 0.00038850626358028303 < 0.0005


In [28]:
ddfm=deep_dyn_fact_mdl

In [29]:
#dir(deep_dyn_fact_mdl)

In [30]:
type(ddfm.factors_filtered)

numpy.ndarray

### factores filtrados

In [32]:
ff=ddfm.factors_filtered
ff.shape

(577, 7)

In [33]:
f_hat_ddfm_idx = pd.DataFrame(ff, columns=["f1", "f2", "f3", "f4", "f5", "f6", "f7"])

In [34]:
f_hat_ddfm_idx.head()

,f1,f2,f3,f4,f5,f6,f7
0,-1.626767,-0.270785,0.537013,0.256650,2.032055,1.324050,-0.814786
1,-4.513472,-2.333861,-3.987064,-4.464952,-6.411350,-3.028942,-0.006983
2,-4.743648,-1.274925,-3.157191,-4.112005,-4.952827,-3.202081,0.419193
3,-9.797295,-8.226314,-7.982015,-8.989313,-10.403632,-7.956421,0.164267
4,-1.451818,0.502306,-0.561069,-0.760574,-0.395649,-2.341089,0.175911


In [36]:
f_hat_ddfm_idx.index = df_train_idx.index
f_hat_ddfm=f_hat_ddfm_idx.reset_index()

In [40]:
f_hat_ddfm.to_csv('/content/drive/MyDrive/D2FM/f_hat_ddfm.csv', index=False)